[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EN/quantlets/EMQA_vol_dynamics/EMQA_vol_dynamics.ipynb)

# EMQA_vol_dynamics
Conditional volatility dynamics -- GARCH conditional vol vs realized (rolling) volatility.
**Output:** `vol_dynamics.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
# Fetch and compute returns
brent = fetch('BZ=F', start='2018-01-01')
log_ret = (np.log(brent / brent.shift(1)).dropna()) * 100

# Realized vol (30-day rolling, annualized)
realized_vol = log_ret.rolling(30).std() * np.sqrt(252)

# GARCH conditional vol
try:
    from arch import arch_model
    am = arch_model(log_ret, vol='Garch', p=1, q=1, dist='normal', mean='Constant')
    res = am.fit(disp='off')
    cond_vol = res.conditional_volatility * np.sqrt(252)  # annualize
except ImportError:
    omega, alpha, beta_ = 0.01, 0.08, 0.90
    n = len(log_ret)
    cond_var = np.zeros(n)
    cond_var[0] = log_ret.var()
    ret_vals = log_ret.values
    for t in range(1, n):
        cond_var[t] = omega + alpha * ret_vals[t-1]**2 + beta_ * cond_var[t-1]
    cond_vol = pd.Series(np.sqrt(cond_var) * np.sqrt(252), index=log_ret.index)

print(f"Mean GARCH vol:     {cond_vol.mean():.2f}%")
print(f"Mean Realized vol:  {realized_vol.mean():.2f}%")

In [ ]:
# 2-panel: Overlaid volatilities, Ratio
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Panel 1 - Both volatilities overlaid
axes[0].plot(realized_vol.index, realized_vol.values, color=COLORS['blue'],
             linewidth=1, label='Realized Vol (30-day rolling)')
axes[0].plot(cond_vol.index, cond_vol.values, color=COLORS['red'],
             linewidth=1, label='GARCH Conditional Vol')
axes[0].set_ylabel('Annualized Volatility (%)')
axes[0].set_title('Conditional vs Realized Volatility - Brent Crude')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), frameon=False, ncol=2)

# Panel 2 - Ratio
common_idx = realized_vol.dropna().index.intersection(cond_vol.dropna().index)
ratio = cond_vol.loc[common_idx] / realized_vol.loc[common_idx]
axes[1].plot(common_idx, ratio.values, color=COLORS['purple'], linewidth=0.8)
axes[1].axhline(1.0, color=COLORS['gray'], linestyle='--', linewidth=1)
axes[1].set_ylabel('GARCH / Realized Ratio')
axes[1].set_xlabel('Date')
axes[1].set_title('Volatility Ratio (GARCH / Realized)')

plt.tight_layout()
save_fig(fig, 'vol_dynamics.pdf')
plt.show()